In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container {width:100% !important; }</style>"))

In [2]:
# Create analysis environment, install libraries and import packages
!pip install pmdarima
!pip install missingno

# Python gets the webpages
import requests
from urllib.request import urlopen
# Creates structured, searchable object
from bs4 import BeautifulSoup  
import os
import math as m
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import json
from datetime import datetime
import datetime as dt
import sys
import seaborn as sns
from bokeh.plotting import figure, show, output_notebook
output_notebook()
from bokeh.layouts import gridplot
# Ignore unnecessary warnings
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import pearsonr
import pmdarima as pmd
from statsmodels.tsa.arima.model import ARIMA
import missingno as msno

%matplotlib inline

Loading BokehJS ...

In [3]:
os.chdir("../assignment")
train = pd.read_csv("train_data.csv")
train.head()

,rumination,grazing,resting,mob_median_grazing,mob_std_grazing,mob_median_resting,mob_std_resting,mob_median_rumination,mob_std_rumination,utc_timestamp,...,pitch_std,farm_average_odba_average,farm_average_odba_std,farm_average_pitch_average,farm_average_pitch_std,farm_average_roll_average,farm_average_roll_std,sickness_type,hours_since_sickness,farm_id
0,12.0,0.0,58.0,0.0,NaN,58.0,NaN,12.0,NaN,2022-08-11 04:00:00+00:00,...,0.0,1403.0,1451.0,-11.0,11.0,12660.0,5230.0,DOWN_COW,117.0,6d60ee09-e8ce-451b-9b31-06afa9c52140
1,34.0,1.0,41.0,2.0,4.0,46.0,6.0,26.0,8.0,2022-08-04 14:00:00+00:00,...,3.0,599.0,578.0,1.0,6.0,10623.0,4189.0,DOWN_COW,-41.0,6d60ee09-e8ce-451b-9b31-06afa9c52140
2,49.0,0.0,49.0,0.0,NaN,49.0,NaN,49.0,NaN,2022-08-06 16:00:00+00:00,...,1.0,446.0,367.0,1.0,5.0,10620.0,3060.0,DOWN_COW,9.0,6d60ee09-e8ce-451b-9b31-06afa9c52140
3,78.0,0.0,81.0,0.0,NaN,81.0,NaN,78.0,NaN,2022-08-08 06:00:00+00:00,...,0.0,883.0,833.0,0.0,7.0,10228.0,4283.0,DOWN_COW,47.0,6d60ee09-e8ce-451b-9b31-06afa9c52140
4,9.0,0.0,51.0,2.0,5.0,42.0,7.0,26.0,8.0,2022-08-04 12:00:00+00:00,...,2.0,765.0,743.0,-1.0,7.0,11033.0,4162.0,DOWN_COW,-43.0,6d60ee09-e8ce-451b-9b31-06afa9c52140


In [3]:
#print("Current working directory:", os.getcwd())
os.chdir("../assignment")
train = pd.read_csv("train_data.csv")
train.head()

,rumination,grazing,resting,mob_median_grazing,mob_std_grazing,mob_median_resting,mob_std_resting,mob_median_rumination,mob_std_rumination,utc_timestamp,...,roll_std,pitch_average,pitch_std,farm_average_odba_average,farm_average_odba_std,farm_average_pitch_average,farm_average_pitch_std,farm_average_roll_average,farm_average_roll_std,farm_id
0,0.0,71.0,0.0,24.0,8.0,26.0,6.0,34.0,9.0,2022-12-02 08:00:00+00:00,...,6513.0,-33.0,10.0,1840.0,1403.0,-16.0,6.0,11920.0,4890.0,36a981ca-0efa-4892-bf07-74bd0210a528
1,54.0,2.0,47.0,0.0,6.0,46.0,6.0,49.0,10.0,2022-12-02 09:00:00+00:00,...,2246.0,16.0,4.0,765.0,562.0,-1.0,4.0,7981.0,3291.0,36a981ca-0efa-4892-bf07-74bd0210a528
2,60.0,0.0,50.0,0.0,2.0,50.0,2.0,36.0,10.0,2022-12-02 10:00:00+00:00,...,1930.0,16.0,3.0,579.0,417.0,-2.0,4.0,9889.0,3541.0,36a981ca-0efa-4892-bf07-74bd0210a528
3,21.0,1.0,51.0,1.0,7.0,49.0,7.0,44.0,10.0,2022-12-02 11:00:00+00:00,...,3173.0,13.0,5.0,1043.0,727.0,-9.0,4.0,9762.0,3635.0,36a981ca-0efa-4892-bf07-74bd0210a528
4,41.0,15.0,32.0,1.0,9.0,46.0,10.0,32.0,11.0,2022-12-02 12:00:00+00:00,...,3757.0,-3.0,6.0,1080.0,748.0,-9.0,4.0,9357.0,3337.0,36a981ca-0efa-4892-bf07-74bd0210a528


In [4]:
# Check the number of rows of the dataset
print("The number of rows:", len(test))

The number of rows: 783494


In [5]:
# Display the column names (variables) in the header
print(test.columns)

Index(['rumination', 'grazing', 'resting', 'mob_median_grazing',
       'mob_std_grazing', 'mob_median_resting', 'mob_std_resting',
       'mob_median_rumination', 'mob_std_rumination', 'utc_timestamp',
       'cattle_id', 'farm_average_rumination', 'farm_average_grazing',
       'farm_average_resting', 'odba_average', 'odba_std', 'roll_average',
       'roll_std', 'pitch_average', 'pitch_std', 'farm_average_odba_average',
       'farm_average_odba_std', 'farm_average_pitch_average',
       'farm_average_pitch_std', 'farm_average_roll_average',
       'farm_average_roll_std', 'farm_id'],
      dtype='object')


In [7]:
# Select rows with the first two cattle ids
specified_cattle_ids = ["092741cd-adea-4d43-9419-06da335e9d46", "1e5849d3-865f-4f11-bc19-b2ba6948ce86"]
selected_rows = test[test['cattle_id'].isin(specified_cattle_ids)]
# Add a new index column to the selected_rows DataFrame
selected_rows = selected_rows.reset_index(drop=False)
# Rename the new index column
selected_rows = selected_rows.rename(columns={'index': 'new_index'})
# Check the number of rows of the dataset
print("The number of rows:", len(selected_rows))

The number of rows: 4798


In [8]:
# Save the selected rows as a new CSV file
selected_rows.to_csv('../assignment/selected_rows.csv')

In [9]:
# Read in the data of the first two cows
os.chdir("../assignment")
select = pd.read_csv("selected_rows.csv")
select.head()

,Unnamed: 0,new_index,rumination,grazing,resting,mob_median_grazing,mob_std_grazing,mob_median_resting,mob_std_resting,mob_median_rumination,...,roll_std,pitch_average,pitch_std,farm_average_odba_average,farm_average_odba_std,farm_average_pitch_average,farm_average_pitch_std,farm_average_roll_average,farm_average_roll_std,farm_id
0,0,0,0.0,71.0,0.0,24.0,8.0,26.0,6.0,34.0,...,6513.0,-33.0,10.0,1840.0,1403.0,-16.0,6.0,11920.0,4890.0,36a981ca-0efa-4892-bf07-74bd0210a528
1,1,1,54.0,2.0,47.0,0.0,6.0,46.0,6.0,49.0,...,2246.0,16.0,4.0,765.0,562.0,-1.0,4.0,7981.0,3291.0,36a981ca-0efa-4892-bf07-74bd0210a528
2,2,2,60.0,0.0,50.0,0.0,2.0,50.0,2.0,36.0,...,1930.0,16.0,3.0,579.0,417.0,-2.0,4.0,9889.0,3541.0,36a981ca-0efa-4892-bf07-74bd0210a528
3,3,3,21.0,1.0,51.0,1.0,7.0,49.0,7.0,44.0,...,3173.0,13.0,5.0,1043.0,727.0,-9.0,4.0,9762.0,3635.0,36a981ca-0efa-4892-bf07-74bd0210a528
4,4,4,41.0,15.0,32.0,1.0,9.0,46.0,10.0,32.0,...,3757.0,-3.0,6.0,1080.0,748.0,-9.0,4.0,9357.0,3337.0,36a981ca-0efa-4892-bf07-74bd0210a528
